In [ ]:
# Path of the file to read

# Create target object and call it y

# Create X

# Split into validation and training data

# Specify Model

# Fit Model

# Make validation predictions and calculate mean absolute error

# Show a prediction result and real data.

In [2]:
# Importing the libraries
import pandas as pd

In [79]:
# Impoirting the dataset
house_prices_dataset_path = "./data/house-prices-advanced-regression-techniques/sample_submission.csv"
house_prices_test_path = "./data/house-prices-advanced-regression-techniques/test.csv"
house_prices_train_path = "./data/house-prices-advanced-regression-techniques/train.csv"
house_data_discription_path = "./data/house-prices-advanced-regression-techniques/data_description.txt"

house_prices = pd.read_csv(house_prices_dataset_path)
house_prices_test = pd.read_csv(house_prices_test_path)
house_prices_train = pd.read_csv(house_prices_test_path)

In [20]:
# Exporting the data (name of columns) to .xlsx file to more convenient work.
df = pd.DataFrame(house_prices_train.columns)
df.to_excel('./auxiliary_files/show_columns.xlsx', sheet_name='Column Names')

In [302]:
class Evaluate_the_data:
    """Checks validation of the data"""
    def __init__(self, data_set, col_name):
        self.data = data_set
        self.col_name = col_name
    
    def is_nan(self):
        # returning number of the NaN cells.
        return pd.isnull(self.data[self.col_name]).value_counts()

    def value_counter(self):
        # returning count number of rows with each unique value of variable.
        return self.data[self.col_name].value_counts(sort=False)

In [300]:
columns_of_dataset = house_prices_test.columns.tolist()

In [303]:
for i in columns_of_dataset:
    test = Evaluate_the_data(house_prices_test, f'{i}')
    
    print("\n" + 25*"=", i, 25*"=")
    
    print(50 * "-")
    print(f"NaN cells:\n{test.is_nan()}")
    print(50 * "-")
    print(f"Counter of unique values:\n{test.value_counter()}")
    


========================= Id =========================
--------------------------------------------------
NaN cells:
False    1459
Name: Id, dtype: int64
--------------------------------------------------
Counter of unique values:
2048    1
2049    1
2050    1
2051    1
2052    1
       ..
2043    1
2044    1
2045    1
2046    1
2047    1
Name: Id, Length: 1459, dtype: int64

========================= MSSubClass =========================
--------------------------------------------------
NaN cells:
False    1459
Name: MSSubClass, dtype: int64
--------------------------------------------------
Counter of unique values:
20     543
30      70
40       2
45       6
50     143
60     276
70      68
75       7
80      60
85      28
90      57
120     95
150      1
160     65
180      7
190     31
Name: MSSubClass, dtype: int64

========================= MSZoning =========================
--------------------------------------------------
NaN cells:
False    1455
True        4
Name: MSZoning

In [ ]:
future = ['MSSubClass', ]